<table style="background-color:#FFFFFF">   
  <tr>     
  <td><img src="https://upload.wikimedia.org/wikipedia/commons/9/95/Logo_EPFL_2019.svg" width="150x"/>
  </td>     
  <td>
  <h1> <b>CS-461: Foundation Models and Generative AI</b> </h1>
  Prof. Charlotte Bunne  
  </td>   
  </tr>
</table>

# 📚 Graded Assignment 1  
### CS-461: Foundation Models and Generative AI - Fall 2025  - Due: October 8, 23:59 CET

Welcome to the first graded assignment!
In this assignment, you will **implement and explore self-supervised learning** on a downsampled subset of the [ImageNet-1k dataset](https://www.image-net.org/), and evaluate how well your model generalizes **both in-distribution and out-of-distribution (OOD)**.  

---

## 🎯 Learning Objectives
By completing this assignment, you will learn to:
- Implement a custom **encoder** and **projection head** for images  
- Experiment with **data augmentations** for self-supervised learning  
- Train a model using a **self-supervised loss**  
- Evaluate learned representations with **k-NN** and **linear probes**  
- Assess **out-of-distribution (OOD) generalization** to unseen classes  
- Save, visualize, and submit results in a reproducible way  

---

## ⚡ Practical Notes
- **Dataset:**  
  - Training: 200 ImageNet classes, 500 images each (100k total)  
  - Validation: 200 ImageNet classes, 50 images each (10k total)  
  - **OOD dataset:** 200 unseen classes, 50 images each (10k total)  
- Use OOD only for **evaluation**, never for training.  
- Checkpoints and evaluation intervals are already set up — your main tasks are to fill in missing functions and customize the model.  
- Some helper utilities (e.g., dataset loaders, probes) are provided in `utils.py`.  

---

👉 **Deliverables:** You will submit:
- Your modified **`models.py`**  
- Trained weights in **`final_model.safetensors`**  
- A short **report.md** (max 500 words) — including **discussion of OOD results**  
- This completed notebook **CS461_Assignment1.ipynb**  

---

⚠️ **Important:** Don’t forget to fill in your **SCIPER number** and **full name** in Section 0, otherwise you will receive **0 points**.  

First, we import packages and set up the device. \
Feel free to add any additional packages you may need.

In [40]:
# Automatically reloads modules when you make changes (useful during development)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
from pathlib import Path
import shutil

import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader
from safetensors.torch import save_model

from torch.amp import autocast, GradScaler

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 🆔 0. SCIPER Number and Name  

⚠️ **IMPORTANT!** ⚠️  
You **must** fill in your **SCIPER number** and **full name** below.  

This is **required for automatic grading**.  
If you do **not** provide this information, you will receive **0️⃣ (zero)** for this assignment. 

In [42]:
SCIPER = "395715"
LAST_NAME = "Kroknes-Gomez"
FIRST_NAME = "Yasmine"

## 1. Datasets & Utilities

- In the following, we will work with a subset of the ImageNet-1k dataset: color images downsampled to 64×64, covering 200 classes.
- The training set contains 500 images per class (100,000 images in total), and the validation set contains 50 images per class (10,000 images in total).
- The Out-Of-Distribution (OOD) datasets contain images from classes not present in the training set. It contains 50 images from 200 different classes (1,000 images in total).
- The purpose of these OOD datasets is to evaluate the generalization capabilities of the learned representations. You should not use it for training.
- During evalution, we will measure your model's performance on another OOD dataset (different from the one provided here), so make sure to not overfit on the provided OOD dataset.

<!-- Let's download/load it and define a default transformation turning a PIL Image into a `torch.tensor` -->
Make sure that you have access to the `/shared/CS461/cs461_assignment1_data/` folder. The folder structure should look like this:
```
cs461_assignment1_data/
└── train.npz
└── val.npz
└── ood.npz
```


Import dataset class and other utilities you developed in previous homeworks:

In [43]:
from utils import ImageDatasetNPZ, default_transform, seed_all
from utils import run_knn_probe, run_linear_probe, extract_features_and_labels

For reproducibility, you can use the provided `seed_all` function to set the random seed for all relevant libraries (Python, NumPy, PyTorch).

In [44]:
seed_all(42)  # For reproducibility, you can use any integer here

You probably want to implement custom data augmentations for the self-supervised learning method you choose. \
Feel free to swap the `default_transform` defined below and create multiple instances of datasets with different transforms.

In [45]:
data_dir = Path('/shared/CS461/cs461_assignment1_data/')

In [46]:
class TwoCropTransform:
    def __init__(self, base_transform):
        self.base = base_transform
    def __call__(self, x):
        return self.base(x), self.base(x)

# two stochastic for train
simclr_train_transform = T.Compose([
    T.ToPILImage(), 
    T.RandomResizedCrop(64, scale=(0.08, 1.0)),
    T.RandomHorizontalFlip(),
    T.RandomApply([T.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
    T.RandomGrayscale(p=0.2),
    T.GaussianBlur(kernel_size=9, sigma=(0.1, 2.0)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

# lighter, deterministic for eval
simclr_eval_transform = T.Compose([
    T.ToPILImage(),
    T.Resize(64),
    T.CenterCrop(64),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

train_dataset = ImageDatasetNPZ(data_dir / 'train.npz', transform=TwoCropTransform(simclr_train_transform))
val_dataset = ImageDatasetNPZ(data_dir / 'val.npz', transform=simclr_eval_transform)

# train "eval" dataset that is single view to build the kNN/linear feature bank
train_eval_dataset = ImageDatasetNPZ(data_dir/'train.npz', transform=simclr_eval_transform)

You can split the provided OOD dataset into a training and validation set using the code below. \
You should not use the training split for actually training your models, but only for evaluation (e.g. kNN or linear probing).

In [47]:
rng = np.random.RandomState(42)
ds_ood = ImageDatasetNPZ(data_dir / 'ood.npz', transform=default_transform)
ood_val_ratio = 0.2
train_mask = rng.permutation(len(ds_ood)) >= int(len(ds_ood) * ood_val_ratio)
ds_oods_train = torch.utils.data.Subset(ds_ood, np.where(train_mask)[0])
ds_oods_val = torch.utils.data.Subset(ds_ood, np.where(~train_mask)[0])

In [48]:
batch_size = 128
num_workers = 4
pin_memory = True
collate_fn = None  # Replace with your custom collate function if needed

In [49]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=True, collate_fn=collate_fn)
val_loader  = DataLoader(val_dataset,  batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory, shuffle=False, collate_fn=collate_fn)

train_eval_loader  = DataLoader(train_eval_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

# 2. Load Your Model

- Load your model from `models.py`.
- You will need to modify the `encoder` and `projection` modules, as the provided template implementation is only a placeholder.
- You SHOULD NOT change the `input_dim`, `input_channels`, and `feature_dim` parameters of the `ImageEncoder` class.
- You can use an existing architecture (e.g., ResNet, ViT) but you SHOULD NOT use any pre-trained weights.

In [50]:
from models import ImageEncoder

model = ImageEncoder().to(device)
model

ImageEncoder(
  (encoder): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d

## 3. Helpers for Training & Evaluation

We suggest you to implement the following helper functions to keep your training and evaluation loops clean and organized. 
- `training_step`: Performs a single training step (forward pass, loss computation, backward pass, optimizer step) and returns the loss value.
- `evaluation_step`: Evaluates the model on the validation dataset and returns the accuracy.

Depending on your specific requirements, you may also want to implement additional utility functions for tasks such as data loading, metric computation, and logging.

As you have seen from previous assignments, loss functions for self-supervised learning objectives can be quite complex. \
Feel free to implement any helper functions you may need to compute the loss.


In [51]:
def training_step(model, batch, optimizer, scaler=None, clip_grad_norm=None, amp_dtype=torch.float16):
    # TODO: Implement the training step
    model.train()
    
    (x1, x2), _ = batch
    device = next(model.parameters()).device
    x1 = x1.to(device, non_blocking=True)
    x2 = x2.to(device, non_blocking=True)

    optimizer.zero_grad(set_to_none=True)
    if scaler is not None:
        with autocast('cuda', dtype=amp_dtype):
            _, z1 = model(x1)
            _, z2 = model(x2)
            loss = custom_loss_function(z1, z2)
        scaler.scale(loss).backward()

        if clip_grad_norm is not None:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)
            
        scaler.step(optimizer)
        scaler.update()

    else:
        _, z1 = model(x1)
        _, z2 = model(x2)
        loss = custom_loss_function(z1, z2)
        loss.backward()

        if clip_grad_norm is not None:
            nn.utils.clip_grad_norm_(model.parameters(), clip_grad_norm)

        optimizer.step()

    return float(loss.item())

In [52]:
def evaluation_step(model, eval_loader, batch, do_linear=True):
    # TODO: Implement the evaluation step
    model.eval()

    x_tr, y_tr = extract_features_and_labels(model, train_eval_loader, normalize=True)
    x_va, y_va = extract_features_and_labels(model, val_loader, normalize=True)

    x_tr, y_tr = x_tr.cpu().numpy(), y_tr.cpu().numpy()
    x_va, y_va = x_va.cpu().numpy(), y_va.cpu().numpy()

    knn_acc = run_knn_probe(x_tr, y_tr, x_va, y_va)
    out = {"knn_accuracy": 100.0 * knn_acc}

    if do_linear:
        lin_acc = run_linear_probe(x_tr, y_tr, x_va, y_va)
        out["linear_accuracy"] = 100.0 * lin_acc
    
    return out

In [53]:
def custom_loss_function(z1, z2, temperature: float = 0.1):
    # TODO: Depend on your training paradigm, implement your custom loss function
    B, D = z1.size()
    z = torch.cat([z1, z2], dim=0)
    sim = (z @ z.T)

    logits = sim / temperature
    diag = torch.eye(2*B, device=z.device, dtype=torch.bool)
    logits = logits.masked_fill(diag, float('-inf'))

    targets = torch.arange(B, device=z.device)
    targets = torch.cat([targets + B, targets], dim=0)

    loss = F.cross_entropy(logits, targets)
    return loss

# 4. Optimizer Configuration

In [54]:
# Feel free to adapt and add more arguments
lr = 1e-3
weight_decay = 5e-2
lr_step_size = 10
lr_gamma = 0.1

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=lr_step_size, gamma=lr_gamma)

# 5. Training Loop

Adapt your training configuration and implement the training loop. \
You probably want to save model checkpoints and evaluate the model on the validation set at regular intervals.

In [56]:
n_epochs = 200  # Adjust the number of epochs as needed
eval_interval = 5  # Evaluate the model every 'eval_interval' epochs
save_interval = 10  # Save the model every 'save_interval' epochs

checkpoints_dir = Path('checkpoints')
if not checkpoints_dir.exists():
    checkpoints_dir.mkdir(parents=True, exist_ok=False)

In [57]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
scaler = GradScaler(enabled=(device.type == "cuda"))

for epoch in tqdm(range(n_epochs)):
    # TODO: Implement the training and evaluation loop
    running_loss = 0.0

    for batch_idx, batch in enumerate(train_loader):
        loss_val = training_step(model, batch, optimizer, scaler)
        running_loss += loss_val

    avg_train_loss = running_loss / max(1, len(train_loader))

    if lr_scheduler is not None:
        lr_scheduler.step()

    # if (epoch + 1) % eval_interval == 0:
    #     model.eval()
    #     with torch.no_grad():
    #         val_stats = evaluation_step(model, val_loader)  # should internally handle device
    #     print(f"Epoch {epoch+1}/{n_epochs} | "
    #           f"Train Loss: {avg_train_loss:.4f} | "
    #           f"Val kNN-5 Acc: {val_stats['knn_accuracy']:.2f}%")

    if (epoch + 1) % eval_interval == 0:
        stats = evaluation_step(model, train_eval_loader, val_loader, do_linear=True)
        line = (f"Epoch {epoch+1}/{n_epochs} | "
                f"Train Loss: {avg_train_loss:.4f} | "
                f"kNN-5: {stats['knn_accuracy']:.2f}%")
        if 'linear_accuracy' in stats:
            line += f" | Linear: {stats['linear_accuracy']:.2f}%"
        print(line)

    if (epoch + 1) % save_interval == 0:
        checkpoint_path = checkpoints_dir / f"model_epoch_{epoch+1}.safetensors"
        save_model(model, checkpoint_path)
        print(f"Model checkpoint saved at {checkpoint_path}")

# Save the final model
final_model_path = checkpoints_dir / 'model_final.safetensors'
save_model(model, final_model_path)

  2%|▎         | 5/200 [10:26<7:28:23, 137.96s/it]

Epoch 5/200 | Train Loss: 4.4382 | kNN-5: 1.52% | Linear: 2.48%


  4%|▍         | 9/200 [17:39<6:04:08, 114.39s/it]hwloc/linux: failed to find sysfs cpu topology directory, aborting linux discovery.


Epoch 10/200 | Train Loss: 3.5836 | kNN-5: 2.44% | Linear: 4.23%


  5%|▌         | 10/200 [21:04<7:31:00, 142.42s/it]

...

Epoch 195/200 | Train Loss: 3.1829 | kNN-5: 2.83% | Linear: 4.79%


100%|█████████▉| 199/200 [6:50:14<01:53, 113.22s/it]hwloc/linux: failed to find sysfs cpu topology directory, aborting linux discovery.


Epoch 200/200 | Train Loss: 3.1793 | kNN-5: 2.92% | Linear: 5.19%


100%|██████████| 200/200 [6:53:29<00:00, 124.05s/it]

Model checkpoint saved at checkpoints/model_epoch_200.safetensors


# 6. Visualize Results

To better understand the performance of your trained model, visualize some results. \
You can visualize:
- Sample images from the validation set along with their predicted labels.
- Training and validation loss curves over epochs.

In [ ]:
# TODO: Visualize some results from your trained model.

# 7. Submission Instructions

You must submit the following files:
- `models.py`: Contains the implementation of your model architecture.
- `final_model.safetensors`: The trained model weights saved in the safetensors format.
- `report.md`: A brief report summarizing your approach, design choices, and results.
- `CS461_Assignment1.ipynb`: The Jupyter notebook containing your code and explanations. Make sure to save your progress before running the cell below.

You will submit your assignment under a single folder named `/home/cs461_assignment1_submission` containing the above files. \
Make sure to replace `<SCIPER>`, `<LAST_NAME>`, and `<FIRST_NAME>` with your actual SCIPER number, last name, and first name respectively. \
The following cell will help you move the files into the submission folder.

In [ ]:
work_dir = Path('.')
output_dir = Path.home() / 'cs461_assignment1_submission'

if not output_dir.exists():
    output_dir.mkdir(parents=True, exist_ok=False)
    
shutil.copy(final_model_path, output_dir / 'final_model.safetensors')
shutil.copy(work_dir / 'models.py', output_dir / 'models.py')
shutil.copy(work_dir / 'CS461_Assignment1.ipynb', output_dir / 'CS461_Assignment1.ipynb')
shutil.copy(work_dir / 'report.md', output_dir / 'report.md')

Check that all required files are present in the submission folder before running the cell below.

In [ ]:
assert SCIPER is not None and LAST_NAME is not None and FIRST_NAME is not None, "Please set your SCIPER, LAST_NAME, and FIRST_NAME variables."

list_of_files = ['final_model.safetensors', 'models.py', 'CS461_Assignment1.ipynb', 'report.md']
files_found = all((output_dir / f).exists() for f in list_of_files)
assert files_found, f"One or more required files are missing in the submission folder: {list_of_files}"


You can test whether your submission folder is appropriately structured by using the `eval.py`:
```bash
python eval.py
```

In [2]:
### Uncomment the line below to run the evaluation script and check your model's performance

# !python eval.py

---
🎉 **Congratulations!**  
You’ve completed Assignment 1. Good luck, and don’t forget to double-check your submission!